In [ ]:
from numpy.typing import NDArray  # trying to be typesafe
import numpy as np  # needed all over the place
import scipy
from scipy.stats import skew
from IPython.display import HTML # in line animations

import os   # file and file path

from itertools import count
import pandas as pd
import matplotlib.pyplot as plt  # for plots

from matplotlib import colors  # not quite sure what fore
from matplotlib.ticker import PercentFormatter  # also not sure, maybe animation..?
from matplotlib.animation import FuncAnimation, FFMpegWriter  # for animations

### Funcitons and classes for Simulations

In [ ]:
class Particles:
    """
    Container for all particles of a simulation with the same attributes and interactions.

    Fields:

        n: (int)
                -> Number of particles 

        m: (float)
                -> Mass of the paricles

        d: (int)
                -> Dimension in which the particles live

        r: (float)
                -> Radius of particles

        x: (NDArray[np.flaot64, shape=(d,n)]),
                -> Array of positions vectors; All positions

        v: (NDArray[np.flaot64, shape=(d,n)])
                -> Array of velocity vectors; All velocities

        a: (NDArray[np.flaot64, shape=d,n])
                -> Array of acceleration vectors; All accelerations
    """
    def __init__(
        self,
        number: int,
        mass: float | int,
        radius: float | int,
        dimensions: int = 2
    ):
        """
        Initiate particles with unpecified positions, velocities, accelerations.
        """
        # constants
        self.n: int = number
        self.m: float = float(mass)
        self.d: int = dimensions
        self.r: float = radius

        self.x: NDArray[np.float64]= np.zeros((dimensions,number))
        self.v: NDArray[np.float64]= np.zeros((dimensions,number))

In [ ]:
def initiate_positions_on_grid(
        number_of_particles: int,
        box: tuple[float|int, float|int]
) -> None:
    """
    Initializes 2D positions on a Grid with even spacing.

    Parameters:
                
        n_particles:(int)
                -> number of particles

        box:(tuple[number,number]) 
                -> box size (in [nm])

    Return:

        (NDArray[float, shpae=(dimension, number of particles)])
                -> list of vectors of positons that are aranged on a grid
    """
    grid_sections = int(np.ceil(np.sqrt(number_of_particles)))+1  # find the number of colums & rows

    # even spacing
    x_spacing = box[0]/grid_sections 
    y_spacing = box[1]/grid_sections
    # makes grid coordinates
    x, y= np.meshgrid(
        np.arange(grid_sections) * x_spacing, 
        np.arange(grid_sections) * y_spacing
    )
    positions= np.array([x.flatten()[:number_of_particles]+x_spacing/2, y.flatten()[:number_of_particles]+y_spacing/2])
    print("init positions\n",positions)  
    return positions

In [ ]:
def initiate_velocities(
        n_particles: int,
        dimension:int,
        velocity: float
) -> NDArray[np.float64]:
    """
    Initiating random velocities.
    """
    velocities=  (np.random.rand(dimension,n_particles) - 0.5)
    velocities= velocity/np.linalg.norm(velocities) * velocities
    print("init velocities\n", velocities)
    return velocities

In [ ]:
def reflecting_boundry_conditions(
        particles :Particles,
        box: tuple[float|int,float|int]
        ) -> None:
    """
    Reflecs particles on the boundries given by the box.

    Parameters:

        particle: (Particle)
                -> particles of the simulation

        box: (tuple[number, number])
                -> box in which the particles are kept 
    """
    for dim in range(particles.d):
            particles.v[dim, :] = np.where(
                    particles.x[dim, :]-particles.r <0,
                    -particles.v[dim, :],
                    particles.v[dim, :]
                )
            particles.v[dim, :] = np.where(
                    particles.x[dim, :] + particles.r> box[1],
                    -particles.v[dim, :],
                    particles.v[dim, :]
                )
    pass

In [ ]:
def iterate(
        particles: Particles,
        box: tuple,
        dt:float
):
    """
    Tterates the particles with elastic collision.
    """
    for i in range(particles.n):
        reflecting_boundry_conditions(particles=particles, box=box) 

        for j in range(i,particles.n):
            r_rel= particles.x[:,i]-particles.x[:,j]
            d_rel= np.linalg.norm(r_rel)
            
            if d_rel-particles.r <= 0 and d_rel!=0:
                r_pro= r_rel/d_rel
                dv= np.dot((particles.v[:,i]-particles.v[:,j]),r_pro) * r_pro
                particles.v[:,i] -= dv
                particles.v[:,j] += dv
        particles.x+= particles.v*dt   

In [ ]:
def simulate(
        particles:Particles,
        box: tuple,
        Time_steps:int,
        dt:float,
        data: NDArray[np.float32]
) -> None:
        """
        Simulates the trajectory of hard spheres & elastic collision.
        """
        data= np.zeros((Time_steps, particles.d, particles.n))
        for t in range(Time_steps):
                iterate(particles=particles,box=box,dt=dt)
                data[t,:,:]=particles.x

In [ ]:
def animate(
        frame,
        x_data: NDArray[np.float64],
        y_data: NDArray[np.float64], 
        scat
):
    """
    Takes x and y data [time, particle]; init funciton for "creat_animation"
    """
    positions = np.c_[x_data[frame, :], y_data[frame,:]]
    scat.set_offsets(positions)
    return scat,

def animation_plot(
    p_radius: float|int,
    box: tuple[float|int]
):
    """
    takes box size and particle radius. Initation for "creat_animation"
    """
    fig, ax = plt.subplots()
    scat = ax.scatter([], [], s=p_radius)  # Scale size for visibility
    ax.set_xlim(box[0])
    ax.set_ylim(box[1])
    ax.set_title("Simulation Visualization")
    ax.set_xlabel("X [nm]")
    ax.set_ylabel("Y [nm]")
    return scat, fig, ax

def creat_animation(
        animation_box: tuple[int,int],
        particle_drawn_radius: float|int,
        x_data: NDArray[np.float64],
        y_data: NDArray[np.float64],
        save_animation: bool= False,
        animation_name: str= "partilce_simulation",
) -> FuncAnimation: 
    """
    x- and y_data has the form of [time, paritcle]. \n
    Saves the animation if save_animation = True.
    """
    scat, fig, ax = animation_plot(p_radius=particle_drawn_radius, box=animation_box)
    anim = FuncAnimation(fig, animate, frames=len(x_data[:,0]), fargs=(x_data, y_data, scat), interval=20, blit=True)
    writer = FFMpegWriter(fps=10, metadata=dict(artist='Dominic Nieder'), bitrate=1800)
    if save_animation:
        anim.save(animation_name+".mp4")
    return anim